The following code is to take a list of MP-IDs and get the gibbs energy of decomposition for each material. 

In [112]:
from mp_api.client import MPRester
from pymatgen.analysis.pourbaix_diagram import PourbaixDiagram, PourbaixPlotter, PourbaixEntry

from api_key import APIKEY

API_KEY = APIKEY

Take list of MP_IDs and get the material entries from the database

from entries, collect the elemental chemsys and composition and create lists with that data

create a new list with the material ids with "gga" added to search through the large list of data that will be generated in the next few cells

In [113]:
materials = ["mp-2657","mp-20782","mp-866101","mp-861502","mp-568392","mp-560328","mp-861942","mp-690687","mp-558763","mp-1229122"]
mp_entries = []

with MPRester(API_KEY) as mpr:
    for mp_id in materials:
        material_entries = mpr.materials.search(material_ids=[mp_id])

        if material_entries:
            mp_entries.append(material_entries[0])

chemsys_list = [entry.chemsys for entry in mp_entries if hasattr(entry, 'chemsys')]
composition_list = [entry.composition_reduced for entry in mp_entries if hasattr(entry, 'composition_reduced')]
materials_gga = [material + "-GGA" for material in materials]
materials_id = materials

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

the elemental list must not contain O (oxegen) or H (hydrogen) to compute the pourbaix data, the remove_o_and_h function removes those from the chemical composisiotn list

In [125]:
print(materials_id)

['mp-2657', 'mp-20782', 'mp-866101', 'mp-861502', 'mp-568392', 'mp-560328', 'mp-861942', 'mp-690687', 'mp-558763', 'mp-1229122']


In [114]:
from pymatgen.core.composition import Composition

def remove_o_and_h(composition):
    new_composition_elements = []
    for element, count in composition.get_el_amt_dict().items():
        if element not in ['O', 'H']:
            new_composition_elements.append(f"{element}{int(count)}")
    new_composition_str = ' '.join(new_composition_elements)
    return Composition(new_composition_str)

filtered_molecules = [remove_o_and_h(molecule) for molecule in composition_list]

the pourbaix charts also requires the elements to be in a percentage form, the calculate_percent_composition looks at the atoms per element in the composition list and converts it to a percentage

In [115]:
from pymatgen.core.composition import Composition

def calculate_percent_composition(molecule):
    total_atoms = sum(molecule.values())
    return {element.symbol: count / total_atoms for element, count in molecule.items()}

percent_compositions = [calculate_percent_composition(molecule) for molecule in filtered_molecules]

the following code takes the chemsys and percent composition and gets the pourbaix entries and converts it to data. some materials do not have a pourbaix entry to error is placed into the list, (lenght of list needs to match in a future function)

In [116]:
pbx_data = []
pbx_entries = []

with MPRester(API_KEY) as mpr:
    for i, x in zip(chemsys_list, percent_compositions):
        try:
            pourbaix_entries = mpr.get_pourbaix_entries(i)
            pbx_entries.append(pourbaix_entries)
            pbx_diagram = PourbaixDiagram(entries=pourbaix_entries, comp_dict=x)
            pbx_data.append(pbx_diagram)
        except Exception as e:
            pbx_data.append('error')

Retrieving ThermoDoc documents:   0%|          | 0/218 [00:00<?, ?it/s]

Retrieving ThermoDoc documents:   0%|          | 0/230 [00:00<?, ?it/s]

Retrieving ThermoDoc documents:   0%|          | 0/169 [00:00<?, ?it/s]

Retrieving ThermoDoc documents:   0%|          | 0/263 [00:00<?, ?it/s]

Retrieving ThermoDoc documents:   0%|          | 0/343 [00:00<?, ?it/s]

...

Retrieving ThermoDoc documents:   0%|          | 0/554 [00:00<?, ?it/s]

spglib: ssm_get_exact_positions failed (attempt=0). (line 115, /tmp/pip-install-58cw25xt/spglib_6207849ab7c64022b27925c813b8e5ef/src/site_symmetry.c).
spglib: ssm_get_exact_positions failed (attempt=1). (line 115, /tmp/pip-install-58cw25xt/spglib_6207849ab7c64022b27925c813b8e5ef/src/site_symmetry.c).
spglib: ssm_get_exact_positions failed (attempt=2). (line 115, /tmp/pip-install-58cw25xt/spglib_6207849ab7c64022b27925c813b8e5ef/src/site_symmetry.c).
spglib: ssm_get_exact_positions failed (attempt=3). (line 115, /tmp/pip-install-58cw25xt/spglib_6207849ab7c64022b27925c813b8e5ef/src/site_symmetry.c).
spglib: ssm_get_exact_positions failed (attempt=4). (line 115, /tmp/pip-install-58cw25xt/spglib_6207849ab7c64022b27925c813b8e5ef/src/site_symmetry.c).
spglib: ssm_get_exact_positions failed. (line 479, /tmp/pip-install-58cw25xt/spglib_6207849ab7c64022b27925c813b8e5ef/src/refinement.c).
spglib: get_bravais_exact_positions_and_lattice failed. (line 336, /tmp/pip-install-58cw25xt/spglib_6207849ab

Retrieving ThermoDoc documents:   0%|          | 0/286 [00:00<?, ?it/s]

Retrieving ThermoDoc documents:   0%|          | 0/200 [00:00<?, ?it/s]

Retrieving ThermoDoc documents:   0%|          | 0/203 [00:00<?, ?it/s]

the pourbaix entry responds with all viable pourbaix diagrams for a chemsys list, the following code searches through the list to get the correct diagram for the material of interest. 

In [117]:
matched_entries = []

for mat_id, entries in zip(materials_gga, pbx_entries):
    matched_entry = next((entry for entry in entries if entry.entry_id == mat_id), None)
    
    if matched_entry:
        matched_entries.append(matched_entry)
    else:
        matched_entries.append(None)

the stability data is then calculated at ph7 and 0V using the entry names and pbx_data

In [118]:
stabilities = []

for i, x in zip(pbx_data, matched_entries):
    try:
        stability = i.get_decomposition_energy(x, 7, 0)
        stabilities.append(stability)
    except Exception as e:
        stabilities.append("error") 


print(stabilities)


[0.037221306666667196, 0.9013717632604736, 'error', 'error', 0.0257185226293003, 1.2820778767603944, 1.2332830795609881, 0.7292489545924903, 0.28738948977272744, 0.5716259503989086]


In [119]:
combined_list = list(zip(materials_id, stabilities))
data_without_error = [item for item in combined_list if item[1] != 'error']
filtered_data = [item for item in data_without_error if float(item[1]) <= 0.5]
mp_identifiers = [item[0] for item in filtered_data]

In [124]:
print("total materials:",len(materials_id))
print("materials without error:",len(data_without_error))
print("stable materials:",len(filtered_data))
print(mp_identifiers)

total materials: 10
materials without error: 8
stable materials: 3
['mp-2657', 'mp-568392', 'mp-558763']
